In [1]:
# run this in your terminal / Anaconda prompt
!pip install pandas numpy rapidfuzz openpyxl xlrd matplotlib seaborn python-Levenshtein


  Using cached rapidfuzz-3.14.1-cp313-cp313-win_amd64.whl.metadata (12 kB)
Using cached rapidfuzz-3.14.1-cp313-cp313-win_amd64.whl (1.5 MB)

   ---------- ----------------------------- 1/4 [rapidfuzz]
   ---------------------------------------- 4/4 [python-Levenshtein]



In [1]:
# 01_data_preprocessing.ipynb - first cell
import os
import json
import re
from pathlib import Path

import numpy as np
import pandas as pd
from rapidfuzz import process, fuzz

# Base paths - change if your structure differs
BASE = Path(r"D:\Complete_Data\nutrition_project")
DATASETS = BASE / "Datasets"
RECIPE_DIR = DATASETS / "recipe_dataset"
NUTR_DIR = DATASETS / "nutrition_dataset"  # folder where USDA csv/xlsx files are
ENV_DIR = DATASETS / "environment_dataset"
COST_DIR = DATASETS / "cost_dataset"
PROCESSED = BASE / "processed_data"

# Make sure output folder exists
PROCESSED.mkdir(parents=True, exist_ok=True)

# Useful print to confirm
print("Paths set. Processed output will be saved to:", PROCESSED)


Paths set. Processed output will be saved to: D:\Complete_Data\nutrition_project\processed_data


In [2]:
# Load full_format_recipes.json
recipe_json_path = RECIPE_DIR / "full_format_recipes.json"
with open(recipe_json_path, "r", encoding="utf-8") as f:
    recipes_raw = json.load(f)

# Convert to DataFrame (top-level fields)
recipes_df = pd.json_normalize(recipes_raw)
print("Recipes rows:", len(recipes_df))
recipes_df.columns[:40]


Recipes rows: 20130


Index(['directions', 'fat', 'date', 'categories', 'calories', 'desc',
       'protein', 'rating', 'title', 'ingredients', 'sodium'],
      dtype='object')

In [3]:
# Create unique recipe_id (index) to track merges later
recipes_df = recipes_df.reset_index().rename(columns={'index':'recipe_id'})

# 1) ingredients.csv: one row per ingredient usage
ingredients_df = recipes_df[['recipe_id','title','ingredients']].explode('ingredients').reset_index(drop=True)
ingredients_df.rename(columns={'title':'recipe_title','ingredients':'ingredient_raw'}, inplace=True)

# 2) nutrition.csv: nutrition objects for each recipe (if present)
if 'nutrition' in recipes_df.columns:
    nutrition_df = pd.json_normalize(recipes_df['nutrition']).reset_index().rename(columns={'index':'recipe_id'})
    # nutrition_df may include keys like calories, protein... keep them
    # If nutrition is nested differently, inspect nutrition_df.head()
else:
    nutrition_df = pd.DataFrame(columns=['recipe_id'])
    
# Save the extracted files
ingredients_csv = RECIPE_DIR / "ingredients.csv"
nutrition_csv = RECIPE_DIR / "nutrition.csv"
ingredients_df.to_csv(ingredients_csv, index=False)
nutrition_df.to_csv(nutrition_csv, index=False)

print("Saved:", ingredients_csv, "rows:", len(ingredients_df))
print("Saved:", nutrition_csv, "rows:", len(nutrition_df))


Saved: D:\Complete_Data\nutrition_project\Datasets\recipe_dataset\ingredients.csv rows: 199106
Saved: D:\Complete_Data\nutrition_project\Datasets\recipe_dataset\nutrition.csv rows: 0


In [4]:
# Try .csv first; if not present, try .xlsx
def smart_read(path_no_ext):
    p_csv = Path(path_no_ext + ".csv")
    p_xlsx = Path(path_no_ext + ".xlsx")
    if p_csv.exists():
        return pd.read_csv(p_csv, low_memory=False)
    elif p_xlsx.exists():
        return pd.read_excel(p_xlsx)
    else:
        raise FileNotFoundError(f"Neither {p_csv} nor {p_xlsx} found.")

# Adjust to your exact folder name where you extracted USDA files
usda_folder = NUTR_DIR / "FoodData_Central_foundation_food_csv_2025-04-24"
food = smart_read(str(usda_folder / "usda_food"))
food_nutrient = smart_read(str(usda_folder / "usda_food_nutrient"))
nutrient_lookup = smart_read(str(usda_folder / "usda_nutrient_lookup"))
food_portion = None
try:
    food_portion = smart_read(str(usda_folder / "usda_food_portion"))
except FileNotFoundError:
    pass

print("USDA: food rows:", food.shape, "food_nutrient rows:", food_nutrient.shape, "nutrients:", nutrient_lookup.shape)


USDA: food rows: (74175, 5) food_nutrient rows: (155243, 11) nutrients: (477, 5)


In [5]:
# Show common nutrient names to find the exact text field
print("Sample nutrients:")
print(nutrient_lookup[['id','name','unit_name']].head(30))

# Find nutrient IDs for energy, protein, fat, carbs (case-insensitive search)
search_names = ['energy','protein','lipid','fat','carbohydrate','carbs','total lipid']
found = {}
for name in search_names:
    matches = nutrient_lookup[nutrient_lookup['name'].str.contains(name, case=False, na=False)]
    if not matches.empty:
        print(f"\nMatches for '{name}':")
        display(matches[['id','name','unit_name']].head(10))
        found[name] = matches

# Typical nutrient names: 'Energy', 'Protein', 'Total lipid (fat)', 'Carbohydrate, by difference'


Sample nutrients:
      id                                  name unit_name
0   2047      Energy (Atwater General Factors)      KCAL
1   2048     Energy (Atwater Specific Factors)      KCAL
2   1001                                Solids         G
3   1002                              Nitrogen         G
4   1003                               Protein         G
5   1004                     Total lipid (fat)         G
6   1005           Carbohydrate, by difference         G
7   1006  Fiber, crude (DO NOT USE - Archived)         G
8   1007                                   Ash         G
9   1008                                Energy      KCAL
10  1009                                Starch         G
11  1010                               Sucrose         G
12  1011                               Glucose         G
13  1012                              Fructose         G
14  1013                               Lactose         G
15  1014                               Maltose         G
16  1015     

id                               name unit_name
0   2047   Energy (Atwater General Factors)      KCAL
1   2048  Energy (Atwater Specific Factors)      KCAL
9   1008                             Energy      KCAL
63  1062                             Energy        kJ


Matches for 'protein':


id              name unit_name
4   1003           Protein         G
54  1053  Adjusted Protein         G


Matches for 'lipid':


...

id                                               name unit_name
5    1004                                  Total lipid (fat)         G
50   1049                                    Solids, non-fat         G
86   1085                                   Total fat (NLEA)         G
258  1257                           Fatty acids, total trans         G
259  1258                       Fatty acids, total saturated         G
291  1291  Fatty acids, other than 607-615, 617-621, 624-...         G
292  1292                 Fatty acids, total monounsaturated         G
293  1293                 Fatty acids, total polyunsaturated         G
318  1318                      Fatty acids, saturated, other         G
319  1319                     Fatty acids, monounsat., other         G


Matches for 'carbohydrate':


id                         name unit_name
6    1005  Carbohydrate, by difference         G
51   1050   Carbohydrate, by summation         G
73   1072          Carbohydrate, other         G
450  2039                Carbohydrates         G


Matches for 'total lipid':


id               name unit_name
5  1004  Total lipid (fat)         G

In [6]:
# Merge to add nutrient names to the mapping
fn = food_nutrient.merge(nutrient_lookup[['id','name','unit_name']], left_on='nutrient_id', right_on='id', how='left')

# Filter to nutrients we care about by name keywords
kw = ['energy','protein','total lipid','carbohydrate']
fn_filtered = fn[fn['name'].str.lower().str.contains('|'.join(kw), na=False)]

# pivot: rows = fdc_id, columns = nutrient name, values = amount
fn_pivot = (fn_filtered
            .pivot_table(index='fdc_id', columns=fn_filtered['name'], values='amount', aggfunc='mean')
            .reset_index())

# Merge pivot into food table
food_nutr = food.merge(fn_pivot, left_on='fdc_id', right_on='fdc_id', how='left')

# Rename common columns to tidy names (if present)
col_renames = {}
for col in food_nutr.columns:
    lc = col.lower()
    if 'energy' in lc:
        col_renames[col] = 'energy_kcal'
    elif 'protein' in lc and 'protein' not in col_renames.values():
        col_renames[col] = 'protein_g'
    elif 'total lipid' in lc or 'fat' in lc:
        col_renames[col] = 'fat_g'
    elif 'carbohydrate' in lc or 'carbs' in lc:
        col_renames[col] = 'carbs_g'
        
food_nutr.rename(columns=col_renames, inplace=True)

print("Food table with nutrients columns:", [c for c in food_nutr.columns if c in col_renames.values()])


Food table with nutrients columns: ['carbs_g', 'carbs_g', 'energy_kcal', 'energy_kcal', 'energy_kcal', 'protein_g', 'fat_g']


In [7]:
# Small helper: normalize text
def normalize_text(s):
    if pd.isna(s): return ""
    s = str(s).lower()
    s = re.sub(r'\([^)]*\)', '', s)          # remove parentheses
    s = re.sub(r'[^a-z0-9\s]', ' ', s)       # remove punctuation
    s = re.sub(r'\s+', ' ', s).strip()       # collapse spaces
    return s

# Clean ingredients from recipe ingredients.csv
ingredients_df['ingredient_clean'] = ingredients_df['ingredient_raw'].astype(str).apply(normalize_text)

# Clean USDA food descriptions
food_nutr['description_clean'] = food_nutr['description'].astype(str).apply(normalize_text)

# Clean cost file items - we'll read cost file below
print("Sample cleaned ingredient:", ingredients_df['ingredient_clean'].sample(5).tolist())


Sample cleaned ingredient: ['1 small garlic clove chopped', '1 1 2 lb gai lan', '1 cup frozen spinach thawed chopped and drained well or spun dry in salad spinner', '2 tablespoons white miso', '1 cup whole wheat flour']


In [8]:
# --- Environment Dataset ---
env_path = ENV_DIR / "Food_Production.csv"

# Auto-detect file type (CSV or Excel)
if env_path.suffix.lower() == ".csv":
    env_df = pd.read_csv(env_path)
else:
    env_df = pd.read_excel(env_path)

print("✅ Environment dataset loaded successfully!")
print("Shape:", env_df.shape)
print("Columns:", env_df.columns.tolist())

# Find a likely column name for the food item
possible_name_cols = [c for c in env_df.columns if 'food' in c.lower() or 'product' in c.lower() or 'item' in c.lower()]
env_name_col = possible_name_cols[0] if possible_name_cols else env_df.columns[0]

# Normalize names for merging
env_df['env_name_clean'] = env_df[env_name_col].astype(str).apply(normalize_text)



# --- Cost Dataset ---
cost_path_csv = COST_DIR / "wfp_food_prices_ind.csv"
cost_path_xlsx = COST_DIR / "wfp_food_prices_ind.xlsx"

# Use whichever exists
if cost_path_csv.exists():
    cost_df = pd.read_csv(cost_path_csv)
elif cost_path_xlsx.exists():
    cost_df = pd.read_excel(cost_path_xlsx)
else:
    raise FileNotFoundError("❌ No cost dataset found in COST_DIR!")

print("\n✅ Cost dataset loaded successfully!")
print("Shape:", cost_df.shape)
print("Columns:", cost_df.columns.tolist())

# Identify item/commodity/food name column
possible_cost_name_cols = [c for c in cost_df.columns if 'item' in c.lower() or 'commodity' in c.lower() or 'product' in c.lower()]
cost_name_col = possible_cost_name_cols[0] if possible_cost_name_cols else cost_df.columns[0]

# Normalize names for merging
cost_df['cost_name_clean'] = cost_df[cost_name_col].astype(str).apply(normalize_text)

# Display first few rows
print("\nSample rows from cost dataset:")
display(cost_df.head())


✅ Environment dataset loaded successfully!
Shape: (43, 23)
Columns: ['Food product', 'Land use change', 'Animal Feed', 'Farm', 'Processing', 'Transport', 'Packging', 'Retail', 'Total_emissions', 'Eutrophying emissions per 1000kcal (gPO₄eq per 1000kcal)', 'Eutrophying emissions per kilogram (gPO₄eq per kilogram)', 'Eutrophying emissions per 100g protein (gPO₄eq per 100 grams protein)', 'Freshwater withdrawals per 1000kcal (liters per 1000kcal)', 'Freshwater withdrawals per 100g protein (liters per 100g protein)', 'Freshwater withdrawals per kilogram (liters per kilogram)', 'Greenhouse gas emissions per 1000kcal (kgCO₂eq per 1000kcal)', 'Greenhouse gas emissions per 100g protein (kgCO₂eq per 100g protein)', 'Land use per 1000kcal (m² per 1000kcal)', 'Land use per kilogram (m² per kilogram)', 'Land use per 100g protein (m² per 100g protein)', 'Scarcity-weighted water use per kilogram (liters per kilogram)', 'Scarcity-weighted water use per 100g protein (liters per 100g protein)', 'Scarcit

C:\Users\Apoorva\AppData\Local\Temp\ipykernel_15220\3981267627.py:29: DtypeWarning: Columns (4,5,12,13) have mixed types. Specify dtype option on import or set low_memory=False.
  cost_df = pd.read_csv(cost_path_csv)



✅ Cost dataset loaded successfully!
Shape: (172018, 14)
Columns: ['date', 'admin1', 'admin2', 'market', 'latitude', 'longitude', 'category', 'commodity', 'unit', 'priceflag', 'pricetype', 'currency', 'price', 'usdprice']

Sample rows from cost dataset:


date      admin1      admin2            market   latitude  longitude  \
0       #date  #adm1+name  #adm2+name  #loc+market+name   #geo+lat   #geo+lon   
1  1994-01-15       Delhi       Delhi             Delhi  28.666667  77.216667   
2  1994-01-15       Delhi       Delhi             Delhi  28.666667  77.216667   
3  1994-01-15       Delhi       Delhi             Delhi  28.666667  77.216667   
4  1994-01-15       Delhi       Delhi             Delhi  28.666667  77.216667   

             category      commodity        unit         priceflag  \
0          #item+type     #item+name  #item+unit  #item+price+flag   
1  cereals and tubers           Rice          KG            actual   
2  cereals and tubers          Wheat          KG            actual   
3  miscellaneous food          Sugar          KG            actual   
4        oil and fats  Oil (mustard)          KG            actual   

          pricetype   currency   price    usdprice cost_name_clean  
0  #item+price+type  #currency  #value  #value+usd       item name  
1            Retail        INR     8.0      0.2545            rice  
2            Retail        INR     5.0       0.159           wheat  
3            Retail        INR    13.5      0.4294           sugar  
4            Retail        INR    31.0       0.986             oil

In [13]:
# Build candidate lists
usda_choices = food_nutr['description_clean'].dropna().unique().tolist()
cost_choices = cost_df['cost_name_clean'].dropna().unique().tolist()
env_choices = env_df['env_name_clean'].dropna().unique().tolist()

# Unique ingredient list to map (speeds up)
unique_ings = ingredients_df['ingredient_clean'].dropna().unique().tolist()

# Matching function with threshold - returns matched string from choices or None
def best_match(name, choices, score_cutoff=75):
    if not name:
        return None, 0
    res = process.extractOne(name, choices, scorer=fuzz.token_set_ratio)
    if res is None:
        return None, 0
    match, score, _ = res
    if score >= score_cutoff:
        return match, score
    return None, score

# Map ingredients -> usda, cost, env
map_rows = []
for ing in unique_ings:
    usda_match, usda_score = best_match(ing, usda_choices, score_cutoff=78)  # stricter for USDA
    cost_match, cost_score = best_match(ing, cost_choices, score_cutoff=75)
    env_match, env_score = best_match(ing, env_choices, score_cutoff=70)
    map_rows.append({'ingredient_clean':ing,
                     'usda_match':usda_match, 'usda_score':usda_score,
                     'cost_match':cost_match, 'cost_score':cost_score,
                     'env_match':env_match, 'env_score':env_score})
ing_map = pd.DataFrame(map_rows)

# Merge mapping back to ingredients_df
ingredients_mapped = ingredients_df.merge(ing_map, on='ingredient_clean', how='left')

# Save mapping for manual inspection and tuning later
ingredients_mapped.to_csv(PROCESSED / "ingredients_mapped.csv", index=False)
print("Saved ingredients mapping to:", PROCESSED / "ingredients_mapped.csv")


Saved ingredients mapping to: D:\Complete_Data\nutrition_project\processed_data\ingredients_mapped.csv


In [10]:
import pandas as pd
import json
from pathlib import Path
import re

# Define base and recipe paths
BASE_DIR = Path(r"D:\Complete_Data\nutrition_project")
RECIPE_DIR = BASE_DIR / "Datasets" / "recipe_dataset"

# Load JSON
json_path = RECIPE_DIR / "full_format_recipes.json"
print("Loading recipe file:", json_path)

with open(json_path, 'r', encoding='utf-8') as f:
    recipes = json.load(f)

# Convert to DataFrame
recipes_df = pd.DataFrame(recipes)
print("✅ Recipes loaded:", recipes_df.shape)
print("Columns:", recipes_df.columns.tolist())

# Normalize text function
def normalize_text(txt):
    if pd.isna(txt):
        return ""
    txt = txt.lower()
    txt = re.sub(r'[^a-z0-9 ]+', '', txt)
    txt = txt.strip()
    return txt


Loading recipe file: D:\Complete_Data\nutrition_project\Datasets\recipe_dataset\full_format_recipes.json
✅ Recipes loaded: (20130, 11)
Columns: ['directions', 'fat', 'date', 'categories', 'calories', 'desc', 'protein', 'rating', 'title', 'ingredients', 'sodium']


In [12]:
# --- STEP 2: Extract Ingredients (Auto-Detect Column Names) ---

# Peek at first few rows to understand available fields
print("Columns found in recipes_df:", recipes_df.columns.tolist())
print("\nSample record:")
print(recipes_df.iloc[0])

# Try to locate the ingredient column automatically
possible_ing_cols = [c for c in recipes_df.columns if 'ingredient' in c.lower()]
if not possible_ing_cols:
    raise KeyError("❌ No ingredient column found in JSON file — please check column names above.")

ing_col = possible_ing_cols[0]
print(f"\n✅ Using '{ing_col}' as ingredient column.")

# Try to locate recipe name/title column automatically
possible_name_cols = [c for c in recipes_df.columns if any(k in c.lower() for k in ['name','title','recipe'])]
name_col = possible_name_cols[0] if possible_name_cols else recipes_df.columns[0]
print(f"✅ Using '{name_col}' as recipe name column.\n")

# Ensure the ingredient column is a list (some datasets store as comma-separated string)
def ensure_list(val):
    if isinstance(val, list):
        return val
    if isinstance(val, str):
        # split by comma or semicolon
        return [v.strip() for v in val.split(',') if v.strip()]
    return []

recipes_df[ing_col] = recipes_df[ing_col].apply(ensure_list)

# Explode ingredients
ingredients_exploded = recipes_df[[name_col, ing_col]].explode(ing_col).dropna(subset=[ing_col])
ingredients_exploded['ingredient_clean'] = ingredients_exploded[ing_col].apply(normalize_text)

# Drop duplicates
ingredients_unique = ingredients_exploded[['ingredient_clean']].drop_duplicates().reset_index(drop=True)

print("✅ Extracted unique ingredients:", ingredients_unique.shape)
ingredients_unique.head()


Columns found in recipes_df: ['directions', 'fat', 'date', 'categories', 'calories', 'desc', 'protein', 'rating', 'title', 'ingredients', 'sodium']

Sample record:
directions     [1. Place the stock, lentils, celery, carrot, ...
fat                                                          7.0
date                                    2006-09-01T04:00:00.000Z
categories     [Sandwich, Bean, Fruit, Tomato, turkey, Vegeta...
calories                                                   426.0
desc                                                        None
protein                                                     30.0
rating                                                       2.5
title                            Lentil, Apple, and Turkey Wrap 
ingredients    [4 cups low-sodium vegetable or chicken stock,...
sodium                                                     559.0
Name: 0, dtype: object

✅ Using 'ingredients' as ingredient column.
✅ Using 'title' as recipe name column.

✅ Extracted u

ingredient_clean
0  4 cups lowsodium vegetable or chicken stock
1                    1 cup dried brown lentils
2            12 cup dried french green lentils
3                      2 stalks celery chopped
4            1 large carrot peeled and chopped

In [13]:
ingredients_mapped = ingredients_unique.copy()
ingredients_mapped['usda_match'] = ingredients_mapped['ingredient_clean']
ingredients_mapped['env_match'] = ingredients_mapped['ingredient_clean']
ingredients_mapped['cost_match'] = ingredients_mapped['ingredient_clean']

print("✅ ingredients_mapped prepared:", ingredients_mapped.shape)
ingredients_mapped.head()


✅ ingredients_mapped prepared: (82133, 4)


ingredient_clean  \
0  4 cups lowsodium vegetable or chicken stock   
1                    1 cup dried brown lentils   
2            12 cup dried french green lentils   
3                      2 stalks celery chopped   
4            1 large carrot peeled and chopped   

                                    usda_match  \
0  4 cups lowsodium vegetable or chicken stock   
1                    1 cup dried brown lentils   
2            12 cup dried french green lentils   
3                      2 stalks celery chopped   
4            1 large carrot peeled and chopped   

                                     env_match  \
0  4 cups lowsodium vegetable or chicken stock   
1                    1 cup dried brown lentils   
2            12 cup dried french green lentils   
3                      2 stalks celery chopped   
4            1 large carrot peeled and chopped   

                                    cost_match  
0  4 cups lowsodium vegetable or chicken stock  
1                    1 cup dried brown lentils  
2            12 cup dried french green lentils  
3                      2 stalks celery chopped  
4            1 large carrot peeled and chopped

In [15]:
import numpy as np
from rapidfuzz import process, fuzz
# ----------------------------
# 3️⃣ Environment Mapping (Fixed for Duplicate Names)
# ----------------------------

print("🔹 Starting environment mapping...")

env_cols_use = [c for c in env_df.columns if any(k in c.lower() for k in ['co2','ghg','emission','water','land'])]
env_cols_use = env_cols_use[:2]
print("✅ Using environment metric columns:", env_cols_use)

env_simple = env_df[['env_name_clean'] + env_cols_use].dropna(subset=['env_name_clean'])
env_simple = env_simple.groupby('env_name_clean', as_index=False).mean(numeric_only=True)

env_lookup = env_simple.set_index('env_name_clean').to_dict(orient='index')

def get_env_metrics(name):
    return pd.Series(env_lookup.get(name, {col: np.nan for col in env_cols_use}))

batch_size = 20000
env_data = []
for start in range(0, len(ingredients_with_nutri), batch_size):
    end = min(start + batch_size, len(ingredients_with_nutri))
    batch = ingredients_with_nutri['env_match'].iloc[start:end].apply(get_env_metrics)
    env_data.append(batch)

env_df2 = pd.concat(env_data, axis=0).reset_index(drop=True)
final_ingredients = pd.concat([ingredients_with_nutri, env_df2], axis=1)

print("✅ Environment mapping done.")


🔹 Starting environment mapping...
✅ Using environment metric columns: ['Land use change', 'Total_emissions']
✅ Environment mapping done.


In [16]:
output_path = PROCESSED / "ingredients_with_nutrients_cost_env.csv"
final_ingredients.to_csv(output_path, index=False)
print(f"✅ Final enriched dataset saved to:\n{output_path}")
print("Shape:", final_ingredients.shape)


✅ Final enriched dataset saved to:
D:\Complete_Data\nutrition_project\processed_data\ingredients_with_nutrients_cost_env.csv
Shape: (82133, 12)


In [17]:
import pandas as pd
import numpy as np
import json
from pathlib import Path

# ----------------------------
# 1️⃣ Paths and File Loading
# ----------------------------
BASE_DIR = Path(r"D:\Complete_Data\nutrition_project")
RECIPE_DIR = BASE_DIR / "Datasets" / "recipe_dataset"
PROCESSED = BASE_DIR / "processed_data"

# Load enriched ingredient dataset
ingredients_df = pd.read_csv(PROCESSED / "ingredients_with_nutrients_cost_env.csv")
print("✅ Enriched ingredients loaded:", ingredients_df.shape)

# Load recipe JSON
json_path = RECIPE_DIR / "full_format_recipes.json"
with open(json_path, 'r', encoding='utf-8') as f:
    recipes = json.load(f)

recipes_df = pd.DataFrame(recipes)
print("✅ Recipes loaded:", recipes_df.shape)
print("Columns:", recipes_df.columns.tolist())

# Detect ingredients column automatically (like before)
possible_ing_cols = [c for c in recipes_df.columns if 'ingredient' in c.lower()]
ing_col = possible_ing_cols[0]
print(f"✅ Using ingredient column: {ing_col}")

# Normalize text function
import re
def normalize_text(txt):
    if pd.isna(txt):
        return ""
    txt = txt.lower()
    txt = re.sub(r'[^a-z0-9 ]+', '', txt)
    txt = txt.strip()
    return txt

# Ensure ingredients are lists
def ensure_list(val):
    if isinstance(val, list):
        return val
    if isinstance(val, str):
        return [v.strip() for v in val.split(',') if v.strip()]
    return []

recipes_df[ing_col] = recipes_df[ing_col].apply(ensure_list)


C:\Users\Apoorva\AppData\Local\Temp\ipykernel_15220\3567528937.py:14: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  ingredients_df = pd.read_csv(PROCESSED / "ingredients_with_nutrients_cost_env.csv")


✅ Enriched ingredients loaded: (82133, 12)
✅ Recipes loaded: (20130, 11)
Columns: ['directions', 'fat', 'date', 'categories', 'calories', 'desc', 'protein', 'rating', 'title', 'ingredients', 'sodium']
✅ Using ingredient column: ingredients


In [18]:
# Expand recipe ingredients
recipes_exploded = recipes_df[['title' if 'title' in recipes_df.columns else ing_col, ing_col]].explode(ing_col)
recipes_exploded = recipes_exploded.rename(columns={ing_col: 'ingredient_raw'})
recipes_exploded['ingredient_clean'] = recipes_exploded['ingredient_raw'].apply(normalize_text)

print("✅ Exploded recipe-ingredient pairs:", recipes_exploded.shape)
recipes_exploded.head()


✅ Exploded recipe-ingredient pairs: (199106, 3)


title  \
0  Lentil, Apple, and Turkey Wrap    
0  Lentil, Apple, and Turkey Wrap    
0  Lentil, Apple, and Turkey Wrap    
0  Lentil, Apple, and Turkey Wrap    
0  Lentil, Apple, and Turkey Wrap    

                                 ingredient_raw  \
0  4 cups low-sodium vegetable or chicken stock   
0                     1 cup dried brown lentils   
0            1/2 cup dried French green lentils   
0                      2 stalks celery, chopped   
0            1 large carrot, peeled and chopped   

                              ingredient_clean  
0  4 cups lowsodium vegetable or chicken stock  
0                    1 cup dried brown lentils  
0            12 cup dried french green lentils  
0                      2 stalks celery chopped  
0            1 large carrot peeled and chopped

In [19]:
# Merge with ingredient-level dataset
recipes_merged = recipes_exploded.merge(
    ingredients_df,
    left_on='ingredient_clean',
    right_on='ingredient_clean',
    how='left'
)

print("✅ Recipes merged with ingredient data:", recipes_merged.shape)
recipes_merged.head()


✅ Recipes merged with ingredient data: (199106, 14)


title  \
0  Lentil, Apple, and Turkey Wrap    
1  Lentil, Apple, and Turkey Wrap    
2  Lentil, Apple, and Turkey Wrap    
3  Lentil, Apple, and Turkey Wrap    
4  Lentil, Apple, and Turkey Wrap    

                                 ingredient_raw  \
0  4 cups low-sodium vegetable or chicken stock   
1                     1 cup dried brown lentils   
2            1/2 cup dried French green lentils   
3                      2 stalks celery, chopped   
4            1 large carrot, peeled and chopped   

                              ingredient_clean  \
0  4 cups lowsodium vegetable or chicken stock   
1                    1 cup dried brown lentils   
2            12 cup dried french green lentils   
3                      2 stalks celery chopped   
4            1 large carrot peeled and chopped   

                                    usda_match  \
0  4 cups lowsodium vegetable or chicken stock   
1                    1 cup dried brown lentils   
2            12 cup dried french green lentils   
3                      2 stalks celery chopped   
4            1 large carrot peeled and chopped   

                                     env_match  \
0  4 cups lowsodium vegetable or chicken stock   
1                    1 cup dried brown lentils   
2            12 cup dried french green lentils   
3                      2 stalks celery chopped   
4            1 large carrot peeled and chopped   

                                    cost_match  fdc_id  energy_kcal  \
0  4 cups lowsodium vegetable or chicken stock     NaN          NaN   
1                    1 cup dried brown lentils     NaN          NaN   
2            12 cup dried french green lentils     NaN          NaN   
3                      2 stalks celery chopped     NaN          NaN   
4            1 large carrot peeled and chopped     NaN          NaN   

   protein_g  fat_g  carbs_g price  Land use change  Total_emissions  
0        NaN    NaN      NaN   NaN              NaN              NaN  
1        NaN    NaN      NaN   NaN              NaN              NaN  
2        NaN    NaN      NaN   NaN              NaN              NaN  
3        NaN    NaN      NaN   NaN              NaN              NaN  
4        NaN    NaN      NaN   NaN              NaN              NaN

In [20]:
# Select relevant numeric columns (nutrients, price, environment metrics)
numeric_cols = recipes_merged.select_dtypes(include=[np.number]).columns.tolist()

# Aggregate by recipe title
recipe_summary = recipes_merged.groupby('title' if 'title' in recipes_merged.columns else recipes_merged.columns[0])[numeric_cols].agg(['mean', 'sum']).reset_index()

# Flatten multi-level columns
recipe_summary.columns = ['_'.join(col).strip('_') for col in recipe_summary.columns.values]

print("✅ Aggregated recipe-level data:", recipe_summary.shape)
recipe_summary.head()


✅ Aggregated recipe-level data: (17775, 15)


title  fdc_id_mean  fdc_id_sum  \
0                           "Adult" Pimiento Cheese           NaN         0.0   
1                              "Blanketed" Eggplant           NaN         0.0   
2  "Bloody Mary" Tomato Toast with Celery and Hor...          NaN         0.0   
3                        "Brown on Blonde" Blondies           NaN         0.0   
4                           "California Roll" Salad           NaN         0.0   

   energy_kcal_mean  energy_kcal_sum  protein_g_mean  protein_g_sum  \
0               NaN              0.0             NaN            0.0   
1               NaN              0.0             NaN            0.0   
2               NaN              0.0             NaN            0.0   
3               NaN              0.0             NaN            0.0   
4               NaN              0.0             NaN            0.0   

   fat_g_mean  fat_g_sum  carbs_g_mean  carbs_g_sum  Land use change_mean  \
0         NaN        0.0           NaN          0.0                   NaN   
1         NaN        0.0           NaN          0.0                   NaN   
2         NaN        0.0           NaN          0.0                   NaN   
3         NaN        0.0           NaN          0.0                   NaN   
4         NaN        0.0           NaN          0.0                   NaN   

   Land use change_sum  Total_emissions_mean  Total_emissions_sum  
0                  0.0                   NaN                  0.0  
1                  0.0                   NaN                  0.0  
2                  0.0                   NaN                  0.0  
3                  0.0                   NaN                  0.0  
4                  0.0                   NaN                  0.0

In [21]:
output_path = PROCESSED / "recipes_master.csv"
recipe_summary.to_csv(output_path, index=False)
print(f"✅ Final recipe-level dataset saved:\n{output_path}")
print("Shape:", recipe_summary.shape)


✅ Final recipe-level dataset saved:
D:\Complete_Data\nutrition_project\processed_data\recipes_master.csv
Shape: (17775, 15)


In [22]:
import pandas as pd
import json
from pathlib import Path

BASE_DIR = Path("D:/Complete_Data/nutrition_project")
DATA_DIR = BASE_DIR / "processed_data"
RAW_RECIPE_PATH = BASE_DIR / "datasets/recipe_dataset/full_format_recipes.json"

# Load your master (aggregated) dataset
recipes_master = pd.read_csv(DATA_DIR / "recipes_master.csv")

# Load the original JSON recipe file to extract ingredients
with open(RAW_RECIPE_PATH, "r", encoding="utf-8") as f:
    raw_recipes = json.load(f)

# Extract relevant fields
raw_recipes_df = pd.DataFrame([
    {
        "title": r.get("title", "").strip(),
        "ingredient_text": ", ".join(r.get("ingredients", []))
    }
    for r in raw_recipes
])

# Normalize titles to match
recipes_master["title_clean"] = recipes_master["title"].str.lower().str.strip()
raw_recipes_df["title_clean"] = raw_recipes_df["title"].str.lower().str.strip()

# Merge ingredients into your master file
recipes_master = recipes_master.merge(
    raw_recipes_df[["title_clean", "ingredient_text"]],
    on="title_clean",
    how="left"
)

# Check how many matched successfully
match_rate = recipes_master["ingredient_text"].notna().mean() * 100
print(f"✅ Matched ingredients for {match_rate:.2f}% of recipes")

# Save updated master for safety layer use
recipes_master.to_csv(DATA_DIR / "recipes_master_with_ingredients.csv", index=False)
print("✅ Saved enriched file:", DATA_DIR / "recipes_master_with_ingredients.csv")


✅ Matched ingredients for 100.00% of recipes
✅ Saved enriched file: D:\Complete_Data\nutrition_project\processed_data\recipes_master_with_ingredients.csv


In [3]:
# =========================================================
# 📘 01_data_preprocessing.ipynb
# Author: Apoorva Sharma
# Purpose: Build clean ingredient-level dataset with nutrients, cost, and environment data
# =========================================================

import pandas as pd
import numpy as np
from pathlib import Path

# ---------------------------------------------------------
# 1️⃣ Define directories
# ---------------------------------------------------------
BASE_DIR = Path("D:/Complete_Data/nutrition_project")
RAW_DIR = BASE_DIR / "raw_data" / "FoodData_Central_sr_legacy_food_csv_2018-04"
OUT_DIR = BASE_DIR / "processed_data"
OUT_DIR.mkdir(parents=True, exist_ok=True)

# ---------------------------------------------------------
# 2️⃣ Load USDA data
# ---------------------------------------------------------
print("📥 Loading USDA datasets...")
food_file = RAW_DIR / "sr_legacy_food.csv"
nutrient_file = RAW_DIR / "food_nutrient.csv"

fdc_food = pd.read_csv(food_file, encoding="latin-1", low_memory=False)
fdc_nutr = pd.read_csv(nutrient_file, encoding="latin-1", low_memory=False)

print(f"✅ Loaded: fdc_food={fdc_food.shape}, fdc_nutr={fdc_nutr.shape}")

# ---------------------------------------------------------
# 3️⃣ Clean and extract relevant fields (auto-detect correct column names)
# ---------------------------------------------------------

# Find the right description column dynamically
possible_desc_cols = [c for c in fdc_food.columns if "desc" in c.lower() or "name" in c.lower()]
desc_col = possible_desc_cols[0] if possible_desc_cols else fdc_food.columns[1]  # fallback

print(f"🧩 Using description column: {desc_col}")

# Keep only relevant columns
fdc_food = fdc_food[['fdc_id', desc_col]].drop_duplicates()
fdc_food.rename(columns={desc_col: 'description'}, inplace=True)

# Clean text
fdc_food['description_clean'] = (
    fdc_food['description']
    .astype(str)
    .str.lower()
    .str.replace(r"[^a-z0-9\s]", "", regex=True)
    .str.strip()
)


# Keep essential nutrient IDs (based on USDA standard IDs)
NUTRIENT_MAP = {
    1008: 'energy_kcal',
    1003: 'protein_g',
    1004: 'fat_g',
    1005: 'carbs_g'
}

fdc_nutr = fdc_nutr[fdc_nutr['nutrient_id'].isin(NUTRIENT_MAP.keys())]
fdc_nutr['nutrient_name'] = fdc_nutr['nutrient_id'].map(NUTRIENT_MAP)

# Pivot nutrient table
nutrient_pivot = (
    fdc_nutr.pivot_table(
        index='fdc_id', columns='nutrient_name', values='amount', aggfunc='mean'
    )
    .reset_index()
)

# ---------------------------------------------------------
# 4️⃣ Merge nutrients with food descriptions
# ---------------------------------------------------------
merged = pd.merge(fdc_food, nutrient_pivot, on='fdc_id', how='left')

# Drop rows missing most nutrients
merged = merged.dropna(subset=['energy_kcal', 'protein_g'], how='all')

# ---------------------------------------------------------
# 5️⃣ Add synthetic cost & environmental impact values
# ---------------------------------------------------------
np.random.seed(42)
merged['price'] = np.random.uniform(0.5, 5.0, size=len(merged))
merged['Land_use_change'] = np.random.uniform(0.1, 3.0, size=len(merged))
merged['Total_emissions'] = np.random.uniform(0.2, 2.5, size=len(merged))

# ---------------------------------------------------------
# 6️⃣ Save Cleaned Data
# ---------------------------------------------------------
output_file = OUT_DIR / "ingredients_with_nutrients_cost_env.csv"
merged.to_csv(output_file, index=False)

print(f"✅ Saved clean dataset to: {output_file}")
print("📊 Final shape:", merged.shape)

# ---------------------------------------------------------
# 7️⃣ Preview sample data
# ---------------------------------------------------------
display(merged.sample(10))


📥 Loading USDA datasets...
✅ Loaded: fdc_food=(7793, 2), fdc_nutr=(644125, 11)
🧩 Using description column: NDB_number
✅ Saved clean dataset to: D:\Complete_Data\nutrition_project\processed_data\ingredients_with_nutrients_cost_env.csv
📊 Final shape: (7793, 10)


fdc_id  description description_clean  carbs_g  energy_kcal  fat_g  \
4391  171903        14242             14242    13.52         54.0   0.10   
389   167901        10214             10214     0.00        121.0   2.59   
7274  174786        25015             25015    50.50        415.0  12.50   
1983  169495        13817             13817     0.97        268.0  21.31   
3884  171396         3992              3992     6.67         66.0   3.60   
6252  173764        16214             16214     1.40         47.0   2.50   
4920  172432        16093             16093    20.91        570.0  47.58   
359   167871        10134             10134     1.50        145.0   5.53   
4965  172477        16436             16436    14.94        147.0   5.84   
492   168004        28360             28360    64.29        446.0  15.63   

      protein_g     price  Land_use_change  Total_emissions  
4391       0.00  2.935715         0.310778         0.760064  
389       22.81  4.130756         0.300071         2.167937  
7274      25.00  2.435792         1.891447         1.236075  
1983      16.98  2.673360         1.658817         1.996105  
3884       1.76  4.487818         1.888964         0.425617  
6252       4.80  2.018647         1.079776         1.096412  
4920      25.09  4.784281         1.696248         1.685032  
359       20.93  3.089634         0.399126         1.456370  
4965      10.62  1.472141         0.718705         1.308791  
492       12.06  1.748215         2.557161         1.990477

In [1]:
# =========================================================
# 📘 01_data_preprocessing.ipynb
# Author: Apoorva Sharma
# Project: AI-Powered Precision Nutrition
# Purpose: Combine and enrich recipe + ingredient data with USDA nutrients
# =========================================================

import pandas as pd
import numpy as np
from pathlib import Path

# ---------------------------------------------------------
# 1️⃣ Setup paths
# ---------------------------------------------------------
BASE_DIR = Path("D:/Complete_Data/nutrition_project")
RAW_DIR = BASE_DIR / "raw_data" / "FoodData_Central_sr_legacy_food_csv_2018-04"
PROCESSED_DIR = BASE_DIR / "processed_data"

PROCESSED_DIR.mkdir(parents=True, exist_ok=True)

RECIPES_PATH = PROCESSED_DIR / "recipes_master.csv"
INGREDIENTS_PATH = PROCESSED_DIR / "ingredients_with_nutrients_cost_env.csv"
USDA_FOOD_PATH = RAW_DIR / "sr_legacy_food.csv"
USDA_NUTR_PATH = RAW_DIR / "food_nutrient.csv"

# ---------------------------------------------------------
# 2️⃣ Load datasets
# ---------------------------------------------------------
recipes = pd.read_csv(RECIPES_PATH)
ingredients = pd.read_csv(INGREDIENTS_PATH)
fdc_food = pd.read_csv(USDA_FOOD_PATH)
fdc_nutr = pd.read_csv(USDA_NUTR_PATH)

print(f"✅ Loaded recipes: {recipes.shape}")
print(f"✅ Loaded ingredients: {ingredients.shape}")
print(f"✅ Loaded: fdc_food={fdc_food.shape}, fdc_nutr={fdc_nutr.shape}")

# ---------------------------------------------------------
# 3️⃣ Clean and extract relevant USDA nutrient info (Robust)
# ---------------------------------------------------------
print(f"🔍 Columns in USDA food file: {list(fdc_food.columns)[:10]}")

# Try to find a description column dynamically
possible_desc_cols = [c for c in fdc_food.columns if any(k in c.lower() for k in ["desc", "food", "name", "title"])]

if len(possible_desc_cols) == 0:
    print("⚠️ No proper description column found — using 'NDB_number' as fallback if available.")
    if "NDB_number" in fdc_food.columns:
        fdc_food["description"] = fdc_food["NDB_number"].astype(str)
    else:
        fdc_food["description"] = fdc_food["fdc_id"].astype(str)
else:
    desc_col = possible_desc_cols[0]
    print(f"✅ Using '{desc_col}' as description column")
    fdc_food.rename(columns={desc_col: "description"}, inplace=True)

# Keep relevant fields
fdc_food = fdc_food[["fdc_id", "description"]].drop_duplicates()
fdc_food["description_clean"] = (
    fdc_food["description"]
    .astype(str)
    .str.lower()
    .str.replace(r"[^a-z\s]", " ", regex=True)
    .str.replace(r"\s+", " ", regex=True)
    .str.strip()
)

# ---------------------------------------------------------
# 4️⃣ Map key nutrients
# ---------------------------------------------------------
CORE_NUTRIENTS = {
    1008: "energy_kcal",
    1003: "protein_g",
    1004: "fat_g",
    1005: "carbs_g",
}

if "nutrient_id" not in fdc_nutr.columns:
    raise KeyError("❌ The USDA nutrient file is missing 'nutrient_id' column!")

fdc_nutr = fdc_nutr[fdc_nutr["nutrient_id"].isin(CORE_NUTRIENTS.keys())].copy()
fdc_nutr["nutrient_name"] = fdc_nutr["nutrient_id"].map(CORE_NUTRIENTS)

fdc_pivot = (
    fdc_nutr.pivot_table(
        index="fdc_id",
        columns="nutrient_name",
        values="amount",
        aggfunc="mean"
    )
    .reset_index()
)

# Merge nutrient + description
usda_merged = pd.merge(fdc_food, fdc_pivot, on="fdc_id", how="left")
print(f"✅ USDA enriched: {usda_merged.shape}")
display(usda_merged.head())

# ---------------------------------------------------------
# 5️⃣ Ensure Ingredient Text Exists in Recipes
# ---------------------------------------------------------
possible_cols = [c for c in recipes.columns if "ingredient" in c.lower()]

if not possible_cols:
    print("⚠️ No ingredient text found directly in recipes — merging from ingredient dataset.")

    # Flexible merge fix
    possible_recipe_keys = [c for c in ingredients.columns if "title" in c.lower() or "recipe" in c.lower()]
    if possible_recipe_keys:
        recipe_key = possible_recipe_keys[0]
        print(f"✅ Found recipe identifier in ingredients: '{recipe_key}'")

        recipe_ingredient_map = (
            ingredients.groupby(recipe_key)["ingredient_clean"]
            .apply(lambda x: ", ".join(x.dropna().astype(str)))
            .reset_index()
            .rename(columns={"ingredient_clean": "ingredient_text"})
        )

        # Merge safely
        if "title" in recipes.columns:
            recipes = recipes.merge(recipe_ingredient_map, left_on="title", right_on=recipe_key, how="left")
        else:
            recipes["ingredient_text"] = recipe_ingredient_map["ingredient_text"]
    else:
        print("⚠️ No recipe-title column found in ingredients file — creating placeholder ingredient_text.")
        recipes["ingredient_text"] = ""
else:
    ingredient_col = possible_cols[0]
    recipes.rename(columns={ingredient_col: "ingredient_text"}, inplace=True)

# Ensure column exists
if "ingredient_text" not in recipes.columns:
    recipes["ingredient_text"] = ""
recipes["ingredient_text"] = recipes["ingredient_text"].fillna("").astype(str)
print("✅ Ingredient text merged or placeholder created.")

# ---------------------------------------------------------
# 6️⃣ Enrich recipes with average nutrient info (Fixed)
# ---------------------------------------------------------

# Detect possible text column in ingredients file
possible_ing_text_cols = [c for c in ingredients.columns if any(k in c.lower() for k in ["ingredient", "name", "desc", "item"])]

if len(possible_ing_text_cols) == 0:
    raise KeyError("❌ No ingredient description/text column found in ingredient dataset.")
else:
    ing_text_col = possible_ing_text_cols[0]
    print(f"✅ Using '{ing_text_col}' as ingredient text column for matching.")
    ingredients["ingredient_clean"] = (
        ingredients[ing_text_col]
        .astype(str)
        .str.lower()
        .str.replace(r"[^a-z\s]", " ", regex=True)
        .str.replace(r"\s+", " ", regex=True)
        .str.strip()
    )

# Detect available nutrient columns
nutr_cols = ["energy_kcal", "protein_g", "fat_g", "carbs_g", "Total_emissions"]
nutr_cols = [c for c in nutr_cols if c in ingredients.columns]

if len(nutr_cols) == 0:
    print("⚠️ No nutrient columns found in ingredient file — skipping enrichment.")
    recipes["energy_kcal_mean"] = np.nan
    recipes["protein_g_mean"] = np.nan
    recipes["fat_g_mean"] = np.nan
    recipes["carbs_g_mean"] = np.nan
    recipes["Total_emissions_mean"] = np.nan
else:
    summary_list = []
    for _, row in recipes.iterrows():
        ingredients_in_recipe = str(row["ingredient_text"]).lower().split(",")
        matched = ingredients[
            ingredients["ingredient_clean"]
            .astype(str)
            .apply(lambda x: any(ing.strip() in x for ing in ingredients_in_recipe))
        ]

        if matched.empty:
            summary_list.append([row["title"], np.nan, np.nan, np.nan, np.nan, np.nan])
        else:
            summary_list.append(
                [
                    row["title"],
                    matched[nutr_cols].mean().to_dict().get("energy_kcal", np.nan),
                    matched[nutr_cols].mean().to_dict().get("protein_g", np.nan),
                    matched[nutr_cols].mean().to_dict().get("fat_g", np.nan),
                    matched[nutr_cols].mean().to_dict().get("carbs_g", np.nan),
                    matched[nutr_cols].mean().to_dict().get("Total_emissions", np.nan),
                ]
            )

    summary_df = pd.DataFrame(
        summary_list,
        columns=[
            "title",
            "energy_kcal_mean",
            "protein_g_mean",
            "fat_g_mean",
            "carbs_g_mean",
            "Total_emissions_mean",
        ],
    )

    # Merge back
    recipes = pd.merge(recipes, summary_df, on="title", how="left")

# ---------------------------------------------------------
# 7️⃣ Save final enriched dataset
# ---------------------------------------------------------
recipes.drop_duplicates(subset="title", inplace=True)

SAVE_PATH = PROCESSED_DIR / "recipes_enriched.csv"
recipes.to_csv(SAVE_PATH, index=False)

print(f"✅ Final enriched dataset saved → {SAVE_PATH}")
print(f"📊 Shape: {recipes.shape}")
display(recipes.head())


✅ Loaded recipes: (17775, 15)
✅ Loaded ingredients: (7793, 10)
✅ Loaded: fdc_food=(7793, 2), fdc_nutr=(644125, 11)
🔍 Columns in USDA food file: ['fdc_id', 'NDB_number']
⚠️ No proper description column found — using 'NDB_number' as fallback if available.
✅ USDA enriched: (7793, 7)


fdc_id description description_clean  carbs_g  energy_kcal  fat_g  \
0  167512       18634                      41.18        307.0  13.24   
1  167513       18635                      53.42        330.0  11.27   
2  167514       18637                      79.80        377.0   3.70   
3  167515       18639                      46.00        232.0   1.80   
4  167516       18932                      41.05        273.0   9.22   

   protein_g  
0       5.88  
1       4.34  
2       6.10  
3       8.00  
4       6.58

⚠️ No ingredient text found directly in recipes — merging from ingredient dataset.
⚠️ No recipe-title column found in ingredients file — creating placeholder ingredient_text.
✅ Ingredient text merged or placeholder created.
✅ Using 'description' as ingredient text column for matching.
✅ Final enriched dataset saved → D:\Complete_Data\nutrition_project\processed_data\recipes_enriched.csv
📊 Shape: (17775, 21)


title  fdc_id_mean  fdc_id_sum  \
0                           "Adult" Pimiento Cheese           NaN         0.0   
1                              "Blanketed" Eggplant           NaN         0.0   
2  "Bloody Mary" Tomato Toast with Celery and Hor...          NaN         0.0   
3                        "Brown on Blonde" Blondies           NaN         0.0   
4                           "California Roll" Salad           NaN         0.0   

   energy_kcal_mean_x  energy_kcal_sum  protein_g_mean_x  protein_g_sum  \
0                 NaN              0.0               NaN            0.0   
1                 NaN              0.0               NaN            0.0   
2                 NaN              0.0               NaN            0.0   
3                 NaN              0.0               NaN            0.0   
4                 NaN              0.0               NaN            0.0   

   fat_g_mean_x  fat_g_sum  carbs_g_mean_x  ...  Land use change_mean  \
0           NaN        0.0             NaN  ...                   NaN   
1           NaN        0.0             NaN  ...                   NaN   
2           NaN        0.0             NaN  ...                   NaN   
3           NaN        0.0             NaN  ...                   NaN   
4           NaN        0.0             NaN  ...                   NaN   

   Land use change_sum  Total_emissions_mean_x  Total_emissions_sum  \
0                  0.0                     NaN                  0.0   
1                  0.0                     NaN                  0.0   
2                  0.0                     NaN                  0.0   
3                  0.0                     NaN                  0.0   
4                  0.0                     NaN                  0.0   

   ingredient_text energy_kcal_mean_y  protein_g_mean_y  fat_g_mean_y  \
0                          220.243118         11.926873     10.687361   
1                          220.243118         11.926873     10.687361   
2                          220.243118         11.926873     10.687361   
3                          220.243118         11.926873     10.687361   
4                          220.243118         11.926873     10.687361   

   carbs_g_mean_y  Total_emissions_mean_y  
0       19.569788                1.373454  
1       19.569788                1.373454  
2       19.569788                1.373454  
3       19.569788                1.373454  
4       19.569788                1.373454  

[5 rows x 21 columns]